In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=505cc4d80fa49e51f55c9df3c7302a0e6cc0021e229ca59befde29bb7e27ea69
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CombineDataFramesExample") \
    .getOrCreate()

In [10]:
FILE_PATHA = "/content/data_fixed.csv"
FILE_PATHB = "/content/pca_ordered_div_cluster.csv"

In [11]:
data = spark.read.csv(FILE_PATHA, header=True, inferSchema=True)
label = spark.read.csv(FILE_PATHB, header=True, inferSchema=True)

In [12]:
label = label.select("cluster")
label.show()

+-------+
|cluster|
+-------+
|      0|
|      1|
|      0|
|      3|
|      2|
|      2|
|      2|
|      3|
|      3|
|      1|
|      3|
|      0|
|      1|
|      3|
|      0|
|      1|
|      2|
|      0|
|      3|
|      2|
+-------+
only showing top 20 rows



In [13]:
label = label.withColumnRenamed("cluster", "labels")
label.show()

+------+
|labels|
+------+
|     0|
|     1|
|     0|
|     3|
|     2|
|     2|
|     2|
|     3|
|     3|
|     1|
|     3|
|     0|
|     1|
|     3|
|     0|
|     1|
|     2|
|     0|
|     3|
|     2|
+------+
only showing top 20 rows



In [14]:
# Thêm một cột index vào DataFrame `data`
data = data.withColumn("index", monotonically_increasing_id())

# Thêm một cột index vào DataFrame `labels_df`
label = label.withColumn("index", monotonically_increasing_id())

# Sử dụng hàm join để kết hợp `data` và `labels_df` dựa trên cột index
data = data.join(label, on="index", how="inner")

# Loại bỏ cột index không cần thiết
data = data.drop("index")

# Hiển thị 10 dòng đầu tiên của DataFrame `data`
data.show(10)


+------+-------+--------+-------+-----+------+----+----+------+----+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+--------+-----------+---+-----------+--------+---------+---------+-----------+------+
|Income|Kidhome|Teenhome|Recency|Wines|Fruits|Meat|Fish|Sweets|Gold|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Response|CustomerFor|Age|Total_Spent|Children|Is_Parent|Education|Living_With|labels|
+------+-------+--------+-------+-----+------+----+----+------+----+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+--------+-----------+---+-----------+--------+---------+---------+-----------+------+
| 58138|      0|       0|     58|  635|    88|

In [16]:
out = "/content/data_for_eval.csv"
data.write.csv(out, header = True)
print("Lưu thành công!!!")

Lưu thành công!!!
